# WordCount for Appstore reviews for popular virtual platforms

To parse AppStore reviews, I use app_store_scraper and iteratively collect data form different app reviews in US

In [14]:
!pip install app_store_scraper

In [15]:
#Import all the nessesary packages

from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json
from wordcloud import WordCloud

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords =stopwords.words('english')

#not all default stopwords parameters catch unessesary words in reviews, so I extend this list with other words
stopwords.extend(['it’s', 'also', 'would','don’t','i’m', 'i’ve']) 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(rc=custom_params, palette = 'pastel')


from pyspark import SparkConf
from pyspark.sql import SparkSession

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladislavcesnokov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Getting data

In [ ]:
### Set AppStore ID and AppStore app name from appstore link
AppStoreDict = {'686449807': 'telegram-messenger', 
                '333903271': 'twitter', 
                '310633997': 'whatsapp-messenger', 
                '389801252': 'instagram',
                '284882215': 'facebook',
                '414478124': 'wechat',
               '874139669': 'signal-private-messenger',
               '447188370': 'snapchat',
               '429047995': 'pinterest',
               '985746746': 'discord-chat-talk-hangout'}

### Create dictionary with links and app names for ease of future iterative work
LocalAppDict = dict()

for key in AppStoreDict:
    app = AppStore(country='us', app_name=AppStoreDict[key], app_id = key)
    
    ###Set the number of reviews
    app.review(how_many=200)
    
    ###Create dataframe
    df = pd.DataFrame(np.array(app.reviews),columns=['review'])
    df2 = df.join(pd.DataFrame(df.pop('review').tolist()))
    
    ### Select only review columns
    review_only = df2['review']
    
    ### Set path for data import
    name = '/Users/vladislavcesnokov/Desktop/HW2 Data/App Store Review '+AppStoreDict[key]+'.csv'
    
    ### Save data in a folder
    review_only.to_csv(name, header=None,index=False)
    LocalAppDict[AppStoreDict[key]] = name

2022-03-28 00:11:52,013 [INFO] Base - Initialised: AppStore('us', 'telegram-messenger', 686449807)
2022-03-28 00:11:52,015 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/telegram-messenger/id686449807
2022-03-28 00:11:58,227 [INFO] Base - [id:686449807] Fetched 100 reviews (100 fetched in total)
2022-03-28 00:12:04,634 [INFO] Base - [id:686449807] Fetched 200 reviews (200 fetched in total)
2022-03-28 00:12:07,038 [INFO] Base - Initialised: AppStore('us', 'twitter', 333903271)
2022-03-28 00:12:07,039 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/twitter/id333903271
2022-03-28 00:12:12,041 [INFO] Base - [id:333903271] Fetched 80 reviews (80 fetched in total)
2022-03-28 00:12:18,455 [INFO] Base - [id:333903271] Fetched 180 reviews (180 fetched in total)
2022-03-28 00:12:19,689 [INFO] Base - [id:333903271] Fetched 200 reviews (200 fetched in total)
2022-03-28 00:12:22,064 [INFO] Base - Initialised: AppStore('us', 'whatsapp-messenger', 31

In [ ]:
LocalAppDict ### Check local app dictionary

## Counting Words

In [ ]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("WordCount")\
    .getOrCreate()

sc=spark.sparkContext

#Remove Punctuation and Transform All Words to Lowercase
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

df = pd.DataFrame()

for key in LocalAppDict:
    path = LocalAppDict[key]
    apps_rdd=sc.textFile(path)
    apps_rdd = apps_rdd.map(lower_clean_str)
    
    #Split sentence into list of words
    apps_rdd=apps_rdd.flatMap(lambda satir: satir.split(" "))
    
    #Exclude whitespaces
    apps_rdd = apps_rdd.filter(lambda x:x!='')
    
    #Count how many times each word occurs
    apps_count=apps_rdd.map(lambda  word:(word,1))
    
    ##Apply ReduceByKey, rank words and exclude stopwords
    apps_count_RBK=apps_count.reduceByKey(lambda x,y:(x+y)).sortByKey()
    apps_count_RBK=apps_count_RBK.map(lambda x:(x[1],x[0]))
    apps_count_RBK = apps_count_RBK.filter(lambda x: x[1] not in stopwords).sortByKey(False)
    deptDF = spark.createDataFrame(apps_count_RBK, schema = ['Count', 'Word'])
    pandasDF = deptDF.toPandas()
    pandasDF['Flag'] = key
    df = pd.concat([df, pandasDF], ignore_index = True)

In [ ]:
df.sort_values('Count', ascending = 0)[:10] #See 10 most frequent words in reviews

## Visualization

### Wordclouds

In [ ]:
comment_words = ''

#Build wordcloud for every app
for val in df['Word']:
    tokens = str(val)
    tokens = val.split()
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.title(label='\n'+ "Wordcloud for all apps"+'\n', fontsize=40, color="Black")
plt.show()
comment_words = ''

In [ ]:
comment_words = ''
for app in df['Flag'].unique():
    targetdf = df[df['Flag'] == app]
    # iterate through the csv file
    for val in targetdf['Word']:
         
        # typecaste each val to string
        tokens = str(val)
     
        # split the value
        tokens = val.split()
         
        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
         
        comment_words += " ".join(tokens)+" "
     
    wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(comment_words)
     
    # plot the WordCloud image                      
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.title(label='\n'+ "Wordcloud for "+app+'\n', fontsize=40, color="Black")
    plt.show()
    comment_words = ''

### Barcharts

In [ ]:
num = 15 #set the number for top-n

In [ ]:
## Build barcharts for every app
for app in df['Flag'].unique():
    targetdf = df[df['Flag'] == app]
    targetdf[:num].sort_values(by = 'Count').plot.barh(x = 'Word', 
                                                y = 'Count', 
                                                figsize=(7,5), 
                                                legend = False, 
                                                title = 'Top-'+str(num)+' most frequent words in '+app+' reviews');